In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value

type Instruction =
  | Mask of string
  | Mem of int64 * int64

let parseInstruction (s:string) =
  if s.StartsWith("mask = ") then
    Mask s.[7..]
  else
    Mem (int64 s.[4..s.IndexOf(']')-1], int64 s.[s.IndexOf("= ")+2..])

let example = 
  """mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0""".Split('\n') |> Seq.map parseInstruction 

let instructions =
  input.Trim().Split('\n') 
  |> Array.map parseInstruction


In [1]:
#!fsharp
open System

let binaryToLong s = Convert.ToInt64(s,2)

let maskValues maskString =
  let bitmask = 
    maskString
    |> Seq.map (function '1' | '0' -> '0' | _ -> '1')
    |> String.Concat
    |> binaryToLong
  let value = 
    maskString
    |> Seq.map (function '1' -> '1' | _ -> '0')
    |> String.Concat
    |> binaryToLong
  (bitmask, value)

let applyMask (bitmask, value) i = i &&& bitmask ||| value  

let runInstruction (memory, mask) = function
  | Mask x -> (memory, maskValues x)
  | Mem (addr, i) -> (memory |> Map.add addr (applyMask mask i), mask)

instructions
|> Seq.fold runInstruction (Map.empty, (0L, 0L))
|> fst
|> Map.toSeq
|> Seq.map snd
|> Seq.reduce (+)


17934269678453

In [1]:
#!fsharp
let maskedAddresses (mask:string) address =
  let ones = mask.Replace('X', '0') |> binaryToLong
  let xs = mask |> Seq.map (function 'X' -> 0 | _ -> 1) |> String.Concat |> binaryToLong
  let baseAddress = address &&& xs ||| ones
  mask
  |> Seq.rev
  |> Seq.indexed
  |> Seq.filter (snd >> (=) 'X')
  |> Seq.map (fst >> ((<<<) 1L))
  |> Seq.fold (fun addrs n -> addrs |> Set.map ((+) n) |> Set.union addrs) (Set.ofList [baseAddress])

let runInstructionV2 (memory, mask) = function
  | Mask m -> (memory, m)
  | Mem (addr, i) -> (maskedAddresses mask addr |> Seq.fold (fun mem a -> Map.add a i mem) memory, mask)

let example2 = 
  """mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1""".Split('\n') 
  |> Seq.map parseInstruction

instructions
|> Seq.fold runInstructionV2 (Map.empty, "") 
|> fst
|> Map.toSeq 
|> Seq.map snd
|> Seq.reduce (+)

3440662844064